In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import pickle
from time import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('data.pkl', 'rb') as f:
    df_train, labels, df_test, test_orders = pickle.load(f)
    
print(df_train.shape)
print(labels.shape)
print(df_test.shape)

(8474661, 21)
(8474661,)
(4833292, 21)


In [3]:
df_train.head()

,order_id,product_id,user_total_orders,user_total_items,total_distinct_items,user_average_days_between_orders,user_average_basket,order_hour_of_day,days_since_prior_order,days_since_ratio,...,department_id,product_orders,product_reorders,product_reorder_rate,UP_orders,UP_orders_ratio,UP_average_pos_in_cart,UP_reorder_rate,UP_orders_since_last,UP_delta_hour_vs_last
0,1187899,17122,11,59,18,19.0,5.363636,8,14.0,0.736842,...,4,13880,9377.0,0.675576,1,0.090909,6.0,0.090909,6,7
1,1187899,196,11,59,18,19.0,5.363636,8,14.0,0.736842,...,7,35791,27791.0,0.776480,10,0.909091,1.4,0.909091,1,0
2,1187899,26405,11,59,18,19.0,5.363636,8,14.0,0.736842,...,17,1214,536.0,0.441516,2,0.181818,5.0,0.181818,7,1
3,1187899,46149,11,59,18,19.0,5.363636,8,14.0,0.736842,...,7,8558,6953.0,0.812456,3,0.272727,3.0,0.272727,1,0
4,1187899,14084,11,59,18,19.0,5.363636,8,14.0,0.736842,...,16,15935,12923.0,0.810982,1,0.090909,2.0,0.090909,10,0


In [4]:
print(labels[:5])

[0 1 1 1 0]


In [5]:
df_test.head()

,order_id,product_id,user_total_orders,user_total_items,total_distinct_items,user_average_days_between_orders,user_average_basket,order_hour_of_day,days_since_prior_order,days_since_ratio,...,department_id,product_orders,product_reorders,product_reorder_rate,UP_orders,UP_orders_ratio,UP_average_pos_in_cart,UP_reorder_rate,UP_orders_since_last,UP_delta_hour_vs_last
0,2774568,17668,13,88,33,12.0,6.769231,15,11.0,0.916667,...,16,2110,1220.0,0.578199,5,0.384615,3.60,0.384615,2,3
1,2774568,44683,13,88,33,12.0,6.769231,15,11.0,0.916667,...,4,22275,11981.0,0.537868,2,0.153846,9.50,0.153846,7,1
2,2774568,48523,13,88,33,12.0,6.769231,15,11.0,0.916667,...,1,5129,2376.0,0.463248,2,0.153846,6.50,0.153846,4,1
3,2774568,21903,13,88,33,12.0,6.769231,15,11.0,0.916667,...,4,241921,186884.0,0.772500,8,0.615385,4.25,0.615385,1,0
4,2774568,14992,13,88,33,12.0,6.769231,15,11.0,0.916667,...,4,29069,16942.0,0.582820,2,0.153846,7.00,0.153846,6,0


In [6]:
f_to_use = ['user_total_orders', 'user_total_items', 'total_distinct_items',
       'user_average_days_between_orders', 'user_average_basket',
       'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
       'aisle_id', 'department_id', 'product_orders', 'product_reorders',
       'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
       'UP_average_pos_in_cart', 'UP_reorder_rate', 'UP_orders_since_last',
       'UP_delta_hour_vs_last']

print('formating for lgb')
d_train = lgb.Dataset(df_train[f_to_use],
                      label=labels,
                      categorical_feature=['aisle_id', 'department_id'])

train_cv = df_train[f_to_use]
# train_cv = df_train[f_to_use].as_matrix()
# labels = np.array(labels)

del df_train

formating for lgb


In [9]:
print(len(train_cv))
print(len(labels))

8474661
8474661


In [14]:
'''from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as st

clf = lgb.LGBMClassifier(
    task='train',
    boosting_type='gbdt',
    objective='binary',
    metric='binary_logloss',
    num_threads=4)

param_dist = {
    'num_leaves': st.randint(2, 31),
    'max_depth': st.randint(-1, 10),
    'min_data_in_leaf': st.randint(1, 20),
    'min_sum_hessian_in_leaf': st.uniform(1e-3, 1),
    'max_bin': st.randint(10, 255)
}

n_iter_search = 20
random_search = RandomizedSearchCV(
    clf, 
    param_distributions=param_dist,
    n_iter=n_iter_search, 
    n_jobs=-1)

start = time()
random_search.fit(train_cv, labels)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)'''

KeyboardInterrupt: 

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

print('light GBM train :-)')
t0 = time()
bst = lgb.train(params, d_train, ROUNDS)
tt = time() - t0
print("Training complete in {} seconds".format(round(tt, 1)))

del d_train

In [ ]:
lgb.plot_importance(bst, figsize=(9,20))

In [ ]:
print('light GBM predict')
t0 = time()
preds = bst.predict(df_test[f_to_use])
tt = time() - t0
print("Prediction complete in {} seconds".format(round(tt, 1)))

df_test['pred'] = preds

TRESHOLD = 0.22  # guess, should be tuned with crossval on a subset of train data

In [ ]:
print(preds[:5])

In [ ]:
df_test.head()

In [ ]:
# Submission
d = dict()
for row in df_test.itertuples():
    # Append all products with predictions greater than the threshold
    if row.pred > TRESHOLD:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

# Append an explicit 'None' value for those orders not found in dict
for order in test_orders.order_id:
    if order not in d:
        d[order] = 'None'

sub = pd.DataFrame.from_dict(d, orient='index')

sub.reset_index(inplace=True)
sub.columns = ['order_id', 'products']
sub.to_csv('sub.csv', index=False)

In [ ]:
sub.head()